## STEP04.Streamlit 앱 생성

Python SDK로 서비스를 쿼리(snowflake Python 패키지 사용)할 수 있습니다. 이 자습서에서는 Streamlit in Snowflake 애플리케이션에서 Python SDK를 사용하는 방법을 보여줍니다.   

먼저 서비스 생성 단계에서 전역 Snowsight UI 역할이 서비스를 생성하는 데 사용된 역할과 동일한지 확인합니다.   

1. Snowsight 에 로그인합니다.

2. 왼쪽 탐색 메뉴에서 **Projects » Streamlit** 를 선택합니다.

3. **+ Streamlit App** 를 선택합니다.

4. 중요: 앱 위치에 대한 cortex_search_tutorial_db 데이터베이스와 public 스키마를 선택합니다.

5. Streamlit in Snowflake 편집기의 왼쪽 창에서 Packages 를 선택하고 snowflake (버전 >= 0.8.0)를 추가하여 애플리케이션에 패키지를 설치합니다.

6. 예제 애플리케이션 코드를 다음 Streamlit 앱으로 바꿉니다.

In [ ]:
# Import python packages
import streamlit as st
from snowflake.core import Root
from snowflake.snowpark.context import get_active_session

# Constants
DB = "cortex_search_tutorial_db"
SCHEMA = "public"
SERVICE = "airbnb_svc"
BASE_TABLE = "cortex_search_tutorial_db.public.airbnb_listings"
ARRAY_ATTRIBUTES = {"AMENITIES"}


def get_column_specification():
    """
    Returns the name of the search column and a list of the names of the attribute columns
    for the provided cortex search service
    """
    session = get_active_session()
    search_service_result = session.sql(f"DESC CORTEX SEARCH SERVICE {DB}.{SCHEMA}.{SERVICE}").collect()[0]
    st.session_state.attribute_columns = search_service_result.attribute_columns.split(",")
    st.session_state.search_column = search_service_result.search_column
    st.session_state.columns = search_service_result.columns.split(",")

def init_layout():
    st.title("Cortex AI Search")
    st.markdown(f"Querying service: `{DB}.{SCHEMA}.{SERVICE}`".replace('"', ''))

def query_cortex_search_service(query, filter={}):
    """
    Queries the cortex search service in the session state and returns a list of results
    """
    session = get_active_session()
    cortex_search_service = (
        Root(session)
        .databases[DB]
        .schemas[SCHEMA]
        .cortex_search_services[SERVICE]
    )
    context_documents = cortex_search_service.search(
        query,
        columns=st.session_state.columns,
        filter=filter,
        limit=st.session_state.limit)
    return context_documents.results

@st.cache_data
def distinct_values_for_attribute(col_name, is_array_attribute=False):
    session = get_active_session()
    if is_array_attribute:
        values = session.sql(f'''
        SELECT DISTINCT value FROM {BASE_TABLE},
        LATERAL FLATTEN(input => {col_name})
        ''').collect()
    else:
        values = session.sql(f"SELECT DISTINCT {col_name} AS VALUE FROM {BASE_TABLE}").collect()
    return [ x["VALUE"].replace('"', "") for x in values ]

def init_search_input():
    st.session_state.query = st.text_input("Query")

def init_limit_input():
    st.session_state.limit = st.number_input("Limit", min_value=1, value=5)

def init_attribute_selection():
    st.session_state.attributes = {}
    for col in st.session_state.attribute_columns:
        is_multiselect = col in ARRAY_ATTRIBUTES
        st.session_state.attributes[col] = st.multiselect(
            col,
            distinct_values_for_attribute(col, is_array_attribute=is_multiselect)
        )

def display_search_results(results):
    """
    Display the search results in the UI
    """
    st.subheader("Search results")
    for i, result in enumerate(results):
        result = dict(result)
        container = st.expander(f"[Result {i+1}]", expanded=True)

        # Add the result text.
        container.markdown(result[st.session_state.search_column])

        # Add the attributes.
        for column, column_value in sorted(result.items()):
            if column == st.session_state.search_column:
                continue
            container.markdown(f"**{column}**: {column_value}")

def create_filter_object(attributes):
    """
    Create a filter object for the search query
    """
    and_clauses = []
    for column, column_values in attributes.items():
        if len(column_values) == 0:
            continue
        if column in ARRAY_ATTRIBUTES:
            for attr_value in column_values:
                and_clauses.append({"@contains": { column: attr_value }})
        else:
            or_clauses = [{"@eq": {column: attr_value}} for attr_value in column_values]
            and_clauses.append({"@or": or_clauses })

    return {"@and": and_clauses} if and_clauses else {}


def main():
    init_layout()
    get_column_specification()
    init_attribute_selection()
    init_limit_input()
    init_search_input()

    if not st.session_state.query:
        return
    results = query_cortex_search_service(
        st.session_state.query,
        filter = create_filter_object(st.session_state.attributes)
    )
    display_search_results(results)


if __name__ == "__main__":
    st.set_page_config(page_title="Cortex AI Search and Summary", layout="wide")
    main()

### Streamlit-in-Snowflake 코드의 주요 구성 요소에 대한 간략한 분석

- **get_column_specification** 은 *DESCRIBE SQL* 쿼리를 사용하여 검색 서비스에서 사용 가능한 특성에 대한 정보를 가져와 Stremalit 상태에 저장합니다.

- **init_layout** 은 페이지의 헤더와 소개를 설정합니다.

- **query_cortex_search_service** 는 Python 클라이언트 라이브러리를 통해 **Cortex Search Service** 쿼리를 처리합니다.

- **create_filter_object** 는 Streamlit 양식에서 선택한 필터 특성을 적절한 오브젝트로 처리하여 Python 라이브러리가 Cortex Search를 쿼리하는 데 사용할 수 있도록 합니다.

- **distinct_values_for_attribute** 는 필터링 가능한 각 특성에 대해 드롭다운 메뉴를 채울 수 있는 값을 결정합니다.

- **init_search_input**, **init_limit_input**, **init_attribute_selection** 는 검색 쿼리에 대한 입력, 결과 수 제한, 특성 필터를 초기화합니다.

- **display_search_results** 는 검색 결과를 결과 페이지에 표시되는 마크다운 요소로 서식을 지정합니다.